# Demystifying Neural Networks 

---



<div style="text-align:right;font-size:0.7em;">scratch.svg</div>

This was a lot of mathematics but not too much code.
Are ANN libraries actually big, in term of lines of code?
To be fair, yes, several libraries have dozens of JVPs and lost of optimizations.
Yet, a full minimal implementation of an ANN library does not need to be big.

We will try a minimal working (buggy) product that can train an ANN.

In [1]:
import pandas as pd
df = pd.read_csv('./pulsars_tuned.csv')
X, y = df.values[:, :-1], df['label'].values
print(X.shape, y.shape)

(3278, 8) (3278,)


Just need to transform the data as previously.

In [2]:
import numpy as np
y = np.c_[y == 0, y == 1].astype(np.float)
print(y.shape)

(3278, 2)


And import what we will need.

In order to avoid implementing our own list inspection
(`autograd` inspects a list if it is differentiating that argument)
we will use the `inspect` package and differentiate against all arguments.

In [3]:
import inspect
from functools import wraps
import operator as op

Just like `autograd` and `pytorch` we will build our own wrapper on `numpy`.
This can be a very thin wrapper, we only need two things from it:

1. As we perform operations using our `numpy` wrapper a graph is built.
2. The array must be aware of the fact that it is an array node.

In [4]:
# class Node (follows)

def drop_wrapper(*args):
    return [a.val if isinstance(a, Node) else a for a in args]

node_functions = [
    np.mean,
    np.log,
    np.tanh,
]


class NumpyWrap(object):

    def __getattr__(self, key):
        fn = getattr(np, key)
        @wraps(fn)
        def np_fn_wrapper(*args, **kwargs):
            fn_args = drop_wrapper(*args)
            fn_values = drop_wrapper(*kwargs.values())
            kwargs = dict(zip(kwargs.keys(), fn_values))
            ret = fn(*fn_args, **kwargs)
            if fn in node_functions:
                return Node(ret, fn, list(args))
            return ret
        return np_fn_wrapper

    def array(self, *args, **kwargs):
        """Avoid numpy bug https://github.com/numpy/numpy/issues/9028"""
        return Node(*args, **kwargs)


npg = NumpyWrap()

Next we need the graph nodes, these require:

1. An increasing ID, so we know the order in which the operations have been performed.
2. A way of keeping their position in the graph, by keeping a link to the parents.
3. Knowledge of what operation resulted in this graph node, so we can find the right JVP.
4. The transpose operation, it is needed for the matrix multiplication JVP.
5. Operator overload of all operations we may use, so we build the graph nodes as we go.
6. Some extras for debugging such as shape or graphviz output.

In [5]:
class Node(object):
    _max_id = 0

    def __new__(cls, val, func=None, parents=None, *args, **kwargs):
        if isinstance(val, Node):
            return val
        obj = super(Node, cls).__new__(cls)
        kwargs['dtype'] = np.float64
        obj.val = np.array(val, **kwargs)
        obj.func = func
        obj.func_name = None
        if func:
            obj.func_name = func.__name__
        if not parents:
            parents = []
        obj.parents = [Node(x) for x in parents]
        obj.grads = None
        Node._max_id += 1
        obj.id = Node._max_id
        return obj

    def add_grad(self, grad):
        if not self.grads:
            self.grads = Node(npg.zeros_like(self))
        self.grads = self.grads + grad

    def __eq__(self, other):
        if not isinstance(other, Node) and not isinstance(other, np.ndarray):
            return False
        return npg.allclose(self, other)

    def __str__(self):
        return str(self.val)

    def __repr__(self):
        return repr(self.val)

    def graph_str(self):
        form = ['N {}'.format(self.id),
                'Value:',
                str(self.val),
                'Fn: {}'.format(self.func_name),
                'Grads:']
        if self.grads:
            form.append(str(self.grads))
        form.append( 'Parents:')
        for p in self.parents:
            form.append(str(p.val))
        return '\n'.join(form)

    def graphviz(self, graph):
        for p in self.parents:
            graph.append(f'    {p.id} -> {self.id}')
            p.graphviz(graph)
        label = self.graph_str().replace('\n', '\\n')
        graph.append(f'    {self.id} [label="{label}"]')

    def digraph(self):
        graph = ['digraph nodes {']
        self.graphviz(graph)
        graph.append('}')
        return '\n'.join(graph)

    def __add__(self, other):
        o = Node(other)
        return Node(self.val + o.val, op.add, [self, o])

    def __radd__(self, other):
        o = Node(other)
        return Node(o.val + self.val, op.add, [o, self])

    def __sub__(self, other):
        o = Node(other)
        return Node(self.val - o.val, op.sub, [self, o])

    def __rsub__(self, other):
        o = Node(other)
        return Node(o.val - self.val, op.sub, [o, self])

    def __neg__(self):
        return Node(-self.val, op.neg, [self])

    def __mul__(self, other):
        o = Node(other)
        return Node(self.val * o.val, op.mul, [self, o])

    def __rmul__(self, other):
        o = Node(other)
        return Node(o.val * self.val, op.mul, [o, self])

    def __truediv__(self, other):
        o = Node(other)
        return Node(self.val / o.val, op.truediv, [self, o])

    def __rtruediv__(self, other):
        o = Node(other)
        return Node(o.val / self.val, op.truediv, [o, self])

    def __pow__(self, other):
        o = Node(other)
        return Node(self.val ** o.val, op.pow, [self, o])

    def __rpow__(self, other):
        o = Node(other)
        return Node(o.val ** self.val, op.pow, [o, self])

    def __matmul__(self, other):
        o = Node(other)
        return Node(self.val @ o.val, op.matmul, [self, o])

    def __rmatmul__(self, other):
        o = Node(other)
        return Node(o.val @ self.val, op.matmul, [o, self])

    @property
    def T(self):
        return Node(self.val.T, np.transpose, [self])

    @property
    def shape(self):
        return self.val.shape

Once we have a graph built we just need something to walk it backwards
and apply the JVPs throughout the nodes.

We use `inspect` to figure out how many arguments a function passed
to `do_grad` has.

In [6]:
def walk_nodes(n):
    graph = [(n.id, n)]
    for p in n.parents:
        graph += walk_nodes(p)
    return list(dict(graph).items())


def do_grad(f, argnums=0, keepgraph=False):
    if 'all' == argnums:
        spec = inspect.getfullargspec(f)
        argnums = list(range(len(spec.args)))
    if isinstance(argnums, int):
        argnums = [argnums]
    @wraps(f)
    def grad_func(*args, **kwargs):
        wrt = {}
        args = list(args)
        for i in argnums:
            wrt[i] = args[i] = Node(args[i])
        ret = f(*args, **kwargs)
        # initial gradient (currently a hack, should implement ones_like)
        ret.add_grad(Node(npg.ones_like(ret.val, dtype=np.float64)))
        nodes = sorted(walk_nodes(ret), key=lambda x: x[0], reverse=True)
        for n_id, n in nodes:
            if n.func:
                vjp = vjps[n.func]
                vjp(n, n.parents)
        grads = [wrt[x].grads for x in argnums]
        if len(grads) == 1:
            grads = grads[0]
        if keepgraph:
            return grads, ret
        else:
            # zero the grads, in case node-arrays exist in the loss
            for _, n in nodes:
                n.grads = []
        return grads
    return grad_func

And the JVPs themselves.

For each operation we may want to differentiate we need a JVP.
JVPs are sometimes tricky to build.
In other words, JVPs are very short pieces of code but may
take one's sleep to get them right.

In [7]:
vjps = {}


def vjp_add(node, parents):
    grad = npg.sum(node.grads)
    for p in parents:
        p.add_grad(node.grads)
vjps[op.add] = vjp_add


def vjp_sub(node, parents):
    grad = npg.sum(node.grads)
    l, r = parents
    l.add_grad(node.grads)
    r.add_grad(-node.grads)
vjps[op.sub] = vjp_sub


def vjp_neg(node, parents):
    grad = npg.sum(node.grads)
    parents[0].add_grad(-node.grads)
vjps[op.neg] = vjp_neg


def vjp_truediv(node, parents):
    grad = npg.sum(node.grads)
    l, r = parents
    l.add_grad(node.grads / r.val)
    r.add_grad(-1 / node.grads**2)
vjps[op.truediv] = vjp_truediv


def vjp_pow(node, parents):
    grad = npg.sum(node.grads)
    l, r = parents
    l.add_grad(node.grads * r.val * l.val**(r.val - 1))
    # log(-x) would give us a complex number,
    # instead we support even powers or positive bases only
    r.add_grad(node.grads * np.log(npg.abs(l.val)) * l.val**r.val)
vjps[op.pow] = vjp_pow


def vjp_matmul(node, parents):
    grad = npg.sum(node.grads)
    l, r = parents
    #l.add_grad(grad * r.val.T)
    l.add_grad(node.grads @ r.T)
    #r.grads.append((grad.T @ l.val).T)
    r.add_grad(l.T @ node.grads)
vjps[op.matmul] = vjp_matmul


def vjp_mean(node, parents):
    grad = npg.sum(node.grads)
    p = parents[0]
    p.add_grad(node.grads / npg.full(p.val.shape, p.val.size / node.val.size))
vjps[np.mean] = vjp_mean


def vjp_tanh(node, parents):
    grad = npg.sum(node.grads)
    p = parents[0]
    p.add_grad(node.grads * 1 - npg.tanh(npg.tanh(p.val)))
vjps[np.tanh] = vjp_tanh


def vjp_transpose(node, parents):
    grad = npg.sum(node.grads)
    p = parents[0]
    p.add_grad(node.grads.T)
vjps[np.transpose] = vjp_transpose

And that's it!  The code above is enough to build and train an ANN.
Note that we did not use either `pytorch` or `autograd` above.

Next we will just build something similar to what we did with `autograd`
and execute and train a small ANN.

In [8]:
def net_exec(l1, b1, l2, b2, l3, b3, X):
    W = [l1, l2, l3]
    Wb = [b1, b2, b3]
    y_hat = X.T
    for w, b in zip(W, Wb):
        y_hat = npg.tanh(w @ y_hat + b)
    return y_hat.T


def netMSE(l1, b1, l2, b2, l3, b3, X, y):
    y_hat = net_exec(l1, b1, l2, b2, l3, b3, X)
    return np.mean(((y_hat - y)**2))


netMSE_grad = do_grad(netMSE, argnums='all')

That should run an ANN, and get its gradients.
Time to generate an ANN.

In [9]:
def layers(neurons):
    weights = []
    for nl, nr in zip(neurons, neurons[1:]):
        weights.append(Node(npg.random.normal(0, 1/npg.sqrt(nr+nl), (nr, nl))))
        weights.append(Node(npg.random.normal(0, 1/npg.sqrt(nr+nl), (nr, 1))))
    return weights


l1, b1, l2, b2, l3, b3 = layers([8, 25, 10, 2])
print(l1.shape, b1.shape, l2.shape, b2.shape, l3.shape, b3.shape)

(25, 8) (25, 1) (10, 25) (10, 1) (2, 10) (2, 1)


First let's see if our ANN does not blow up.

In [10]:
print(net_exec(l1, b1, l2, b2, l3, b3, X))

[[ 0.00169501 -0.04085595]
 [-0.73685352  0.2662274 ]
 [-0.61084144  0.39672799]
 ...
 [ 0.42612149 -0.14190078]
 [-0.05999345  0.04135883]
 [-0.72195061  0.28300735]]


Good, the ANN works, we can build the gradient evaluator.

Since we are differentiating against every argument we will need to write all of them.
This would be easy to add some syntactic sugar on top but we will keep things simple.
Now, the bias terms are broadcasted during the ANN execution.
Yet, that also means that their gradients have a broadcasted format.
We will need to aggregate them back.

In [11]:
l1g, b1g, l2g, b2g, l3g, b3g, Xg, yg = netMSE_grad(l1, b1, l2, b2, l3, b3, X, y)
b1g = npg.sum(b1g, axis=1)[:, np.newaxis]
b2g = npg.sum(b2g, axis=1)[:, np.newaxis]
b3g = npg.sum(b3g, axis=1)[:, np.newaxis]
print(l1g.shape, b1g.shape, l2g.shape, b2g.shape, l3g.shape, b3g.shape)

(25, 8) (25, 1) (10, 25) (10, 1) (2, 10) (2, 1)


There are still a handful of bugs in the code above,
notably the decisions about when to use our graph based `numpy` wrapper
and when to use plain `numpy` are not well polished yet.
This means that the code below is not very well optimized
and may result in quite a high memory usage to keep the graphs.

That said, it should be enough to train the ANN at hand.

In [12]:
netMSE_grad = do_grad(netMSE, argnums='all', keepgraph=True)

l1, b1, l2, b2, l3, b3 = layers([8, 25, 10, 2])
learning_rate = 0.01
batch = 200
for i in range(1000):
    idx = np.random.randint(0, len(y), batch)
    X_sample, y_sample = X[idx], y[idx]
    grads, ret = netMSE_grad(l1, b1, l2, b2, l3, b3, X_sample, y_sample)
    l1g, b1g, l2g, b2g, l3g, b3g, Xg, yg = drop_wrapper(*grads)
    b1g = np.sum(b1g, axis=1)[:, np.newaxis]
    b2g = np.sum(b2g, axis=1)[:, np.newaxis]
    b3g = np.sum(b3g, axis=1)[:, np.newaxis]
    weights = [l1, b1, l2, b2, l3, b3]
    grads = [l1g, b1g, l2g, b2g, l3g, b3g]
    for j in range(len(weights)):
        weights[j] = weights[j] - grads[j]*learning_rate
        weights[j].grads = None
    if (i+1)%50 == 0:
        print(np.mean(ret.val - y_sample)**2)
    l1, b1, l2, b2, l3, b3 = [Node(x.val) for x in weights]
    del ret

/home/grochmal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in log
/home/grochmal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in multiply


7.412255729886451e-12
0.13689817240517962
0.06241615721135874
0.002149219034025239
0.004899996619739444
3.880736005494303e-11
0.052894994260030574
0.0014191994010394834
0.021268060862543087
0.0048914141911825335
1.0929177878328146e-09
0.08958589488746166
0.009997062117765432
0.10239715203196285
0.08999982994497342
0.028724729671944272
4.813469634255193e-05
0.13676038838447618
0.016888634301367087
0.048396510451469234


Let's do a quick check whether we identified any pulsars.

In [13]:
y_hat = npg.argmax(net_exec(l1, b1, l2, b2, l3, b3, X), axis=1)
print(sum(y_hat))

1554


And finally see the performance of our ANN from scratch.

In [14]:
y_hat = npg.argmax(net_exec(l1, b1, l2, b2, l3, b3, X), axis=1)
y_true = df['label'].values
print(sum(y_hat[y_true == 1] == y_true[y_true == 1])/sum(y_true == 1))
print(sum(y_hat[y_true == 0] == y_true[y_true == 0])/sum(y_true == 0))
print(sum(y_hat == y_true)/len(y_true))

0.8505186089078707
0.9023794996949359
0.8764490543014033


In summary, the code needed to build an ANN is not very extensive.
The mathematics behind it may be complicated at times but the
actual code is reasonably short and sweet.